In [8]:
import requests
from lxml import etree
import re
import json
import pandas as pd

In [9]:
rw_csv = pd.read_csv('rw_csv.csv')

In [10]:
falcon_url = "https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20FalconFeedsIO/view/json"

# fetch the RSS feed
response = requests.get(falcon_url)
rss_json = response.json()

In [11]:
rw_list = []
for item in rss_json['items']:
    if "#ransomware" in item['title']:
        tweet = item['title'].replace('\n',' ')
        link = item['url']
        ransomware = tweet.split('#ransomware')[0]
        date_published = item['date_published'][:10]
        date_published = pd.to_datetime(date_published).strftime('%Y-%m-%d')
        pattern = r"added (.*?) to their victim list"
        try:
            victim = re.search(pattern, tweet).group(1)
            # if "," in victim:
            #     victim = victim.split(',')[0]
            rw_list.append({'victim': victim,
            'ransomware': ransomware,
            'date_published':date_published,
            'link':link})
        except:
            next
                # print(f'Ransomware: {ransomware}\nVictim: {victim}\nLink: {link}\nDate: {date_published}')
        
# rw_df = pd.DataFrame(rw_list)
# rw_df

In [12]:
threatmon_url = 'https://www.inoreader.com/stream/user/1004920918/tag/rw_monitor%3A%20threatmon/view/json'

# fetch the RSS feed
response = requests.get(threatmon_url)
rss_json = response.json()

In [13]:
# rw_list = []
for item in rss_json['items']:
    if "Actor :" in item['title']:
        tweet = item['title'].replace('\n',' ')
        link = item['url']
        ransomware = tweet.split(' Victim :')[0].replace("Actor : ",'').strip()
        date_published = pd.to_datetime(item['date_published'][:10]).strftime('%Y-%m-%d').strip()
        try:
            victim = tweet.split(' Victim : ')[1].split(" Date")[0].strip()
            # print(f'{ransomware} | {victim} | {date_published}')
            rw_list.append({'victim': victim,
            'ransomware': ransomware,
            'date_published':date_published,
            'link':link})
        except:
            next
rw_df = pd.DataFrame(rw_list)
rw_df

,victim,ransomware,date_published,link
0,"Medicalodges, Inc (medicalodges.com), a health...",Karakurt,2023-04-12,https://twitter.com/FalconFeedsio/status/16461...
1,"David S. Brown Enterprises, Ltd (davidsbrown.c...",BlackBasta,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
2,TVH,LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
3,"Conseil de l'Entente (conseildelentente.org), ...",LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
4,"Servex US (servex-us.com),",LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
5,"Medmark Insurance Brokerage (medmark.eg),",LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
6,"ROBOVIC Inc (robovic.com),",LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
7,Iskandar Regional Development Authority (irda....,LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
8,"Comacchio SpA (comacchio.com),",LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...
9,Group Lease Public Company Limited (grouplease...,LockBit,2023-04-12,https://twitter.com/FalconFeedsio/status/16460...


In [14]:
df_combined = pd.concat([rw_df,rw_csv],ignore_index=True)
df_combined.drop_duplicates('link',inplace=True)
rw_df.to_csv('rw_csv.csv', mode='w', header=True, index=False)